# Welcome to the Sagemaker Immersion Day for ISVs



## SageMaker Pipelines Lab
### Overview

**Amazon SageMaker Pipelines**, a new capability of Amazon SageMaker that makes it easy for data scientists and engineers to build, automate, and scale end to end machine learning pipelines. SageMaker Pipelines is a native workflow orchestration tool for building ML pipelines that take advantage of direct Amazon SageMaker integration. Three components improve the operational resilience and reproducibility of your ML workflows: pipelines, model registry, and projects. These workflow automation components enable you to easily scale your ability to build, train, test, and deploy hundreds of models in production, iterate faster, reduce errors due to manual orchestration, and build repeatable mechanisms.

SageMaker projects introduce MLOps templates that automatically provision the underlying resources needed to enable CI/CD capabilities for your ML development lifecycle. You can use a number of built-in templates or create your own custom template (https://docs.aws.amazon.com/sagemaker/latest/dgsagemaker-projects-templates-custom.html). You can use SageMaker Pipelines independently to create automated workflows; however, when used in combination with SageMaker projects, the additional CI/CD capabilities are provided automatically. The following screenshot shows how the three components of SageMaker Pipelines can work together in an example SageMaker project.

![Overview](img/image1.png)



This lab focuses on using an MLOps template to bootstrap your ML project and establish a CI/CD pattern from sample code. We show how to use the built-in build, train, and deploy project template as a base for a customer churn classification example. This base template enables CI/CD for training ML models, registering model artifacts to the model registry, and automating model deployment with manual approval and automated testing.

## MLOps Template for building, training and deploying models

We start by taking a detailed look at what AWS services are launched when this build, train, and deploy MLOps template is launched. Later, we discuss how to modify the skeleton for a custom use case.

In SageMaker Studio, you can now choose the **Projects** menu on the **Components and registries** menu.

![Overview](img/image12.png)

Once you choose Projects, click on Create project as below:

![Overview](img/image13.png)

On the projects page, you can launch a preconfigured SageMaker MLOps template. For this lab, we choose MLOps template for model building, training, and deployment and click on Select project template

![Overview](img/image14.png)

In the next page provide Project Name and short Description and select Create Project.

![Overview](img/image15.png)

The project will take a while to be created.

![Overview](img/image15bis.png)

Launching this template starts a model building pipeline by default, and while there is no cost for using SageMaker Pipelines itself, you will be charged for the services launched. Cost varies by Region. A single run of the model build pipeline in us-east-1 is estimated to cost less than $0.50. Models approved for deployment incur the cost of the SageMaker endpoints (test and production) for the Region using an ml.m5.large instance.

After the project is created from the MLOps template, the following architecture is deployed.

![Overview](img/image16.png)



Included in the architecture are the following AWS services and resources:

* The MLOps templates that are made available through SageMaker projects are provided via an AWS Service Catalog portfolio that automatically gets imported when a user enables projects on the Studio domain.

* Two repositories are added to AWS CodeCommit:

    * The first repository provides scaffolding code to create a multi-step model building pipeline including the following steps: data processing, model training, model evaluation, and conditional model registration based on accuracy. As you can see in the pipeline.py file, this pipeline trains a linear regression model using the XGBoost algorithm on the well-known UCI Abalone dataset. This repository also includes a build specification file, used by AWS CodePipeline and AWS CodeBuild to run the pipeline automatically.

    * The second repository contains code and configuration files for model deployment, as well as test scripts required to pass the quality gate. This repo also uses CodePipeline and CodeBuild, which run an AWS CloudFormation template to create model endpoints for staging and production.

* Two CodePipeline pipelines:

    * The ModelBuild pipeline automatically triggers and runs the pipeline from end to end whenever a new commit is made to the ModelBuild CodeCommit repository.

    * The ModelDeploy pipeline automatically triggers whenever a new model version is added to the model registry and the status is marked as Approved. Models that are registered with Pending or Rejected statuses aren’t deployed.

* An Amazon Simple Storage Service(Amazon S3) bucket is created for output model artifacts generated from the pipeline.

* SageMaker Pipelines uses the following resources:

    * This workflow contains the directed acyclic graph (DAG) that trains and evaluates our model. Each step in the pipeline keeps track of the lineage and intermediate steps can be cached for quickly re-running the pipeline. Outside of templates, you can also create pipelines using the SDK.

    * Within SageMaker Pipelines, the SageMaker model registry tracks the model versions and respective artifacts, including the lineage and metadata for how they were created. Different model versions are grouped together under a model group, and new models registered to the registry are automatically versioned. The model registry also provides an approval workflow for model versions and supports deployment of models in different accounts. You can also use the model registry through the boto3 package.

* Two SageMaker endpoints:
    * After a model is approved in the registry, the artifact is automatically deployed to a staging endpoint followed by a manual approval step.
    * If approved, it’s deployed to a production endpoint in the same AWS account.
    
    

All SageMaker resources, such as training jobs, pipelines, models, and endpoints, as well as AWS resources listed in this lab, are automatically tagged with the project name and a unique project ID tag.


# Looking at the Seed Code for Custom Use Case

After your project has been created, the architecture described earlier is deployed and the visualization of the pipeline is available on the Pipelines drop-down menu within SageMaker Studio.

To modify the sample code from this launched template, we first need to clone the CodeCommit repositories to our local SageMaker Studio instance. From the list of projects, choose the one that was just created. On the Repositories tab, you can select the hyperlinks to locally clone the CodeCommit repos.

![repos](img/image18.png)

Once both repositories have been cloned you should see the following:
![repos](img/jma2.png)

## ModelBuild Repo:

The ModelBuild repository contains the code for preprocessing, training, and evaluating the model. The sample code trains and evaluates a model on [the UCI Abalone dataset](https://archive.ics.uci.edu/ml/datasets/abalone). We can modify these files to solve our own customer churn use case. See the following code:

![code](img/image19.png)


 

## Execute the next few steps to Customize your MLOps env

Select the file browser. On the top right:


![repos](img/jma3.png)


Go to your project Folder. In the modelbuild/pipelines/abalone folder.

![repos](img/jma4.png)


Find the xgb_train.set_hyperparameters:

![repos](img/jma5.png)

   
Modify the hyperparameters for instance : num_round=30

![repos](img/jma6.png)


Save the file

# ModelDeploy repo:

The ModelDeploy repository contains the AWS CloudFormation buildspec for the deployment pipeline. We don’t make any modifications to this code because it’s sufficient for our customer churn use case. It’s worth noting that model tests can be added to this repo to gate model deployment. See the following code:

    ├── build.py
    ├── buildspec.yml
    ├── endpoint-config-template.yml
    ├── prod-config.json
    ├── README.md
    ├── staging-config.json
    └── test
    ├── buildspec.yml
    └── test.py

# Triggering a pipeline run

Committing these changes to the CodeCommit repository (easily done on the Studio source control tab) triggers a new pipeline run, because an Amazon EventBridge event monitors for commits. After a few moments, we can monitor the run by choosing the pipeline inside the SageMaker project.

1. To commit the changes, navigate to the Git Section on the left panel and follow the steps in the screenshot below;
    * Stage all changes
    * Commit the changes by providing a Summary and your Name and an email address
    * Push the changes.

**Make sure you stage the Untracked changes as well.**
![files to replace](img/image24.png)


## Navigate to Pipelines by selecting it from the Sagemaker drop down menu.

### Double click on the pipeline name on the left hand pane

### IMPORTANT: Wait until you see a second pipeline executing before proceeding
Under execution the following screenshot shows our pipeline details. Wait for all piplines to complete successfully
![files to replace](img/image25.png)

3. If you double click on the executing pipelines, the steps of the pipeline will appear. You will be able to monitor the step that is currently running.
![files to replace](img/image26.png)

![files to replace](img/image27.png)


### Next, go to **Model registry** from Sagemaker drop down menu to inspect the newly built model by double clicking on Model group name on the left hand pane. 
![files to replace](img/modelReg.png)

Double click on the model version to inspect the metadata attached to the model artifacts. E.g. click on the Model quality tab to see performance metrics.
![files to replace](img/image28.png)

5. Click on the Update Status button (top right) to approve the model for deployment or right click on the model version to update the status.
![files to replace](img/image29.png)

![files to replace](img/image30.png)

![files to replace](img/image30-2.png)


You can then go to **Endpoints** in the SageMaker menu.
![files to replace](img/image30-3.png)

You will see a staging endpoint being created.
![files to replace](img/image30-4.png)

After a while the endpoint will be listed with the **InService** status.
![files to replace](img/image30-5.png)


### Deploy the endpoint into production
You need to put your "DevOps Team" hat and go to CodePipeline.
![files to replace](img/image30-6.png)


Click on the modeldeploy pipeline which is currently in progress.
![files to replace](img/image30-7.png)


At the end of the DeployStaging phase, you need to manually approve the deployment.
![files to replace](img/image30-8.png)


Once it is done you will see the production endpoint being deployed in the SageMaker Endpoints.
![files to replace](img/image30-9.png)



After a while the endpoint will also be InService.


![files to replace](img/image31.png)




# Multi tenancy

Lets create a second training dataset for our second customer:

In [ ]:
import boto3
import sagemaker
import os
!aws s3 cp s3://sagemaker-servicecatalog-seedcode-us-east-1/dataset/abalone-dataset.csv ./
    
region = boto3.Session().region_name
role = sagemaker.get_execution_role()
default_bucket = sagemaker.session.Session().default_bucket()

### Create 2 distinct datasets, one per tenant

In [ ]:
# Tenant 1
prefix = 'sagemaker/DEMO-xgboost-churn-cust1'
RawData = boto3.Session().resource('s3')\
.Bucket(default_bucket).Object(os.path.join(prefix, 'data/RawData.csv'))\
.upload_file('./abalone-dataset.csv')

s3sourcecust1=os.path.join("s3://",default_bucket, prefix, 'data/RawData.csv')
print(s3sourcecust1)

In [ ]:
# Tenant 2
prefix = 'sagemaker/DEMO-xgboost-churn-cust2'
RawData = boto3.Session().resource('s3')\
.Bucket(default_bucket).Object(os.path.join(prefix, 'data/RawData.csv'))\
.upload_file('abalone-dataset.csv')

s3sourcecust2=os.path.join("s3://",default_bucket, prefix, 'data/RawData.csv')
print(s3sourcecust2)

## Execute the pipeline with a different training dataset


In [ ]:
myPipelineArn='change-with-your-pipeline-arn'
sm_client=boto3.client('sagemaker')
sts_client=boto3.client('sts')
account_id = sts_client.get_caller_identity()["Account"]

In [ ]:
# Tenant 1 pipeline
response = sm_client.start_pipeline_execution(
    PipelineName=myPipelineArn,
    PipelineParameters=[
        {
            'Name': 'InputDataUrl',
            'Value': s3sourcecust1
        },
    ],
    PipelineExecutionDescription='Tenant1'
)

In [ ]:
# Tenant 2 pipeline
response = sm_client.start_pipeline_execution(
    PipelineName=myPipelineArn,
    PipelineParameters=[
        {
            'Name': 'InputDataUrl',
            'Value': s3sourcecust2
        },
    ],
    PipelineExecutionDescription='Tenant2'
)

### Review multi-tenant pipeline execution

Select Pipelines from the Sagemaker drop down menu and double click on the pipeline name on the left hand pane and confirm execution of 2 instances of the pipeline, one for each tenant

# Conclusion

In this lab we have walked through how a data scientist can modify a preconfigured MLOps template for their own modeling use case. Among the many benefits is that the changes to the source code can be tracked, associated metadata can be tied to trained models for deployment approval, and repeated pipeline steps can be cached for reuse. To learn more about SageMaker Pipelines, check out the website and the documentation.